# Подготовка модели распознавания рукописных букв и цифр

In [25]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import cv2

from torchvision.datasets import EMNIST
from torch.utils.data import random_split
from torch.utils.data import DataLoader

In [2]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

# Data Preparation

In [3]:
dataset = EMNIST('data/', 'balanced', download=False)

In [4]:
for i in range(11, 22):
    img, lbl = dataset[i]
    print(lbl, img.size)
    display(img)

43 (28, 28)


15 (28, 28)


13 (28, 28)


24 (28, 28)


12 (28, 28)


...

17 (28, 28)


3 (28, 28)


40 (28, 28)


42 (28, 28)


14 (28, 28)


In [5]:
dataset = EMNIST('data/', 'balanced', download=False, transform=transforms.ToTensor())

In [6]:
type(dataset)

torchvision.datasets.mnist.EMNIST

In [7]:
# Загрузка словаря соответствий
label_mapping = {}
with open('emnist-balanced-mapping.txt', 'r') as file:
    for line in file:
        label, char = line.split()
        label_mapping[int(label)] = chr(int(char))

In [8]:
label_mapping

{0: '0',
 1: '1',
 2: '2',
 3: '3',
 4: '4',
 5: '5',
 6: '6',
 7: '7',
 8: '8',
 9: '9',
 10: 'A',
 11: 'B',
 12: 'C',
 13: 'D',
 14: 'E',
 15: 'F',
 16: 'G',
 17: 'H',
 18: 'I',
 19: 'J',
 20: 'K',
 21: 'L',
 22: 'M',
 23: 'N',
 24: 'O',
 25: 'P',
 26: 'Q',
 27: 'R',
 28: 'S',
 29: 'T',
 30: 'U',
 31: 'V',
 32: 'W',
 33: 'X',
 34: 'Y',
 35: 'Z',
 36: 'a',
 37: 'b',
 38: 'd',
 39: 'e',
 40: 'f',
 41: 'g',
 42: 'h',
 43: 'n',
 44: 'q',
 45: 'r',
 46: 't'}

In [9]:
dataset

Dataset EMNIST
    Number of datapoints: 112800
    Root location: data/
    Split: Train
    StandardTransform
Transform: ToTensor()

In [10]:
generator = torch.Generator().manual_seed(42)

train, val, test = random_split(dataset, [0.7, 0.2, 0.1], generator=generator)
print(f'Размер тренировочной выборки: {train.__len__()}')
print(f'Размер валидационной выборки: {val.__len__()}')
print(f'Размер тестовой выборки: {test.__len__()}')

Размер тренировочной выборки: 78960
Размер валидационной выборки: 22560
Размер тестовой выборки: 11280


# Modeling

In [11]:
batch_size = 64
epochs = 10
lr = 1e-3
m = 0.9

In [12]:
# Data loader
dataloader_train = DataLoader(
    train, 
    batch_size=batch_size, 
    shuffle=True, 
    drop_last=True
)

dataloader_val = DataLoader(
    val, 
    batch_size=batch_size, 
    shuffle=False, 
    drop_last=True
)

dataloader_test = DataLoader(
    test, 
    batch_size=batch_size, 
    shuffle=False, 
    drop_last=True
)

In [13]:
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()

        self.activation = nn.ReLU()
        self.conv_1 = nn.Conv2d(
            in_channels=in_ch,
            out_channels=out_ch,
            kernel_size=3,
            padding=1
        )
        self.bn1 = nn.BatchNorm2d(out_ch)

        self.conv_2 = nn.Conv2d(
            in_channels=out_ch,
            out_channels=out_ch,
            kernel_size=3,
            padding=1
        )
        self.bn2 = nn.BatchNorm2d(out_ch)

    def forward(self, x):
        y = self.activation(self.bn1(self.conv_1(x)))
        y = self.bn2(self.conv_2(y))
        return self.activation(x + y)

In [14]:
class EMNIST_CNN(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        self.conv_1 = nn.Conv2d(1, 16, 3, 1)
        self.conv_2 = nn.Conv2d(16, 32, 3, 1)
        self.block_1 = ConvBlock(32, 32)
        self.pooling_1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.block_2 = ConvBlock(32, 32)
        self.pooling_2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.activation = nn.ReLU()
        self.flat = nn.Flatten()
        self.linear_1 = nn.Linear(1152, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.linear_2 = nn.Linear(512, 47)

    def forward(self, x):
        y = self.conv_1(x) # 16, 28, 28
        y = self.conv_2(y) # 32, 28, 28
        y = self.pooling_1(self.block_1(y)) 
        y = self.pooling_2(self.block_2(y)) 

        y = self.flat(y)
        y = self.activation(self.linear_1(y))
        y = self.bn1(y)
        y = self.activation(self.linear_2(y))
        return y

In [15]:
def accuracy(labels, preds):
    predictions = torch.argmax(preds, dim=1)
    correct = (labels == predictions).sum().numpy()
    return correct / len(labels)

In [14]:
model = EMNIST_CNN()
loss_fn = nn.CrossEntropyLoss()
model = model.to(device)
loss_fn = loss_fn.to(device)

opimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=m)

In [15]:
best_mean_accuracy = 0
for epoch in range(epochs):
    loss_val = 0

    model.train()

    for img, label in dataloader_train:
        img = img.to(device)
        label = label.to(device)
        opimizer.zero_grad()
        label = nn.functional.one_hot(label, 47).float()
        pred = model.forward(img)
        loss = loss_fn(pred, label)
        loss.backward()
        opimizer.step()
        loss_val += loss.item()
    print(f'Epoch {epoch+1}; loss: {loss_val / len(dataloader_train)}')

    model.eval()
    accuracies = []

    for img, label in dataloader_val:
        img = img.to(device)
        label = label.to(device)
        y_pred = model(img)
        accuracies.append(accuracy(label.cpu(), y_pred.cpu()))

    mean_accuracy = np.mean(np.array(accuracies))
    if mean_accuracy > best_mean_accuracy:
        best_mean_accuracy = mean_accuracy
        torch.save(model, 'EMNIST_CNN_best.pth')
    
    print(f'accuracy on validation: {np.mean(np.array(accuracies))}')

Epoch 1; loss: 0.8755571210374701
accuracy on validation: 0.8508966619318182
Epoch 2; loss: 0.4230241534532035
accuracy on validation: 0.8691850142045454
Epoch 3; loss: 0.3636764312965156
accuracy on validation: 0.8757102272727273
Epoch 4; loss: 0.3327684442105568
accuracy on validation: 0.8780628551136364
Epoch 5; loss: 0.3110421671496594
accuracy on validation: 0.876953125
Epoch 6; loss: 0.2937392464749038
accuracy on validation: 0.8843661221590909
Epoch 7; loss: 0.2798436731039656
accuracy on validation: 0.8854758522727273
Epoch 8; loss: 0.26834248622693574
accuracy on validation: 0.8857865767045454
Epoch 9; loss: 0.2587197158555141
accuracy on validation: 0.8814364346590909
Epoch 10; loss: 0.2509949989344952
accuracy on validation: 0.8859641335227273


# Evaluation

In [16]:
model_ckpt = torch.load('myapp/model.ckpt')
trained_model = EMNIST_CNN()
trained_model.load_state_dict(model_ckpt['model_state_dict'])
trained_model.eval()
trained_model = trained_model.to(device)

In [17]:
val_acc = []
for img, label in dataloader_val:
    img = img.to(device)
    label = label.to(device)
    y_pred = trained_model(img)
    val_acc.append(accuracy(label.cpu(), y_pred.cpu()))
print(f'accuracy on test: {np.mean(np.array(val_acc))}')

accuracy on test: 0.8859641335227273


In [18]:
test_acc = []
for img, label in dataloader_test:
    img = img.to(device)
    label = label.to(device)
    y_pred = trained_model(img)
    test_acc.append(accuracy(label.cpu(), y_pred.cpu()))
print(f'accuracy on test: {np.mean(np.array(test_acc))}')

accuracy on test: 0.8858309659090909


In [35]:
#torch.save({'model_state_dict': trained_model.state_dict()}, os.path.join('myapp', 'model.ckpt'))